In [1]:
import pandas as pd
import logging

In [2]:

df = pd.read_csv('/kaggle/input/breakhis/Folds.csv')
df1 = df[df["fold"] == 1].copy()  # Use copy to avoid modifying the original DataFrame
df1.rename(columns={"filename": "path"}, inplace=True)

for index, row in df1.iterrows():
    df1.at[index, "name"] = row["path"].split('/')[8]
    df1.at[index, "class"] = row["path"].split('/')[3]
    df1.at[index, "slide_id"] = row["path"].split('/')[8].split("-")[2]
    df1.at[index, "tumor_id"] = row["path"].split('/')[8].split("-")[0].split("_")[2]

print(df1.tail())

      fold  mag   grp                                               path  \
7904     1  400  test  BreaKHis_v1/histology_slides/breast/malignant/...   
7905     1  400  test  BreaKHis_v1/histology_slides/breast/malignant/...   
7906     1  400  test  BreaKHis_v1/histology_slides/breast/malignant/...   
7907     1  400  test  BreaKHis_v1/histology_slides/breast/malignant/...   
7908     1  400  test  BreaKHis_v1/histology_slides/breast/malignant/...   

                               name      class slide_id tumor_id  
7904  SOB_M_PC-14-15704-400-031.png  malignant    15704       PC  
7905  SOB_M_PC-14-15704-400-032.png  malignant    15704       PC  
7906  SOB_M_PC-14-15704-400-033.png  malignant    15704       PC  
7907  SOB_M_PC-14-15704-400-034.png  malignant    15704       PC  
7908  SOB_M_PC-14-15704-400-035.png  malignant    15704       PC  
